In [1]:
import requests
import re
from bs4 import BeautifulSoup
import json

In [2]:
url = "https://www.youtube.com/channel/UCLNBEt_42kYuX7fgZiubgXQ"
playlists_url = url + "/playlists"
resp = requests.get(playlists_url)
data_soup = BeautifulSoup(resp.text, 'html.parser')

channel_url = url
channel_name = data_soup.select_one('link[itemprop="name"]')["content"]
channel_id = url.replace("https://www.youtube.com/channel/","")

playlistId_pattern = r'playlistId":"(.*?)",'
playlistId_list = re.findall(playlistId_pattern, resp.text)
playlistId_list = list(set(playlistId_list))

# data_soup
# "#header #channel-name #text.ytd-channel-name"

In [3]:
playlistId = "PLI6pJZaOCtF0T_pQ3gv37BmdZabLA9xVX"
yt_list_url = f"https://www.youtube.com/playlist?list={playlistId}"
resp = requests.get(yt_list_url)


In [16]:



data_dict_list = []
for playlist_id in playlistId_list[:1]:
    yt_list_url = f"https://www.youtube.com/playlist?list={playlist_id}"
    resp = requests.get(yt_list_url)
    data_soup = BeautifulSoup(resp.text, 'html.parser')
    playlist_title = data_soup.select_one('meta[property="og:title"]')["content"]

    videoId_pattern = r'"videoId":"(.*?)",'
    videoId_list = re.findall(videoId_pattern, resp.text)
    video_id_list = list(set(videoId_list))
    print(video_id_list)

    data_soup_video_list = []
    for video_id in video_id_list[:1]:
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        print(video_url)
        resp = requests.get(video_url)
        resp.text
        data_soup = BeautifulSoup(resp.text, 'html.parser')
        data_soup_video_list.append(data_soup)

        data_dict = {
            "channel_name": channel_name,
            "channel_url": channel_url,
            "channel_id": channel_id,
            "playlist_id": playlist_id,
            "playlist_title": playlist_title,
            "data_soup_video_list": data_soup_video_list
        }
        data_dict_list.append(data_dict)

# print(data_dict_list)

['8DcJxx5jpFQ', 'E_x4al5INak', 'gZQjUrFW948', 'd3O3ju1l6vI', 'FFFFzEUdsX8', 'T3b8btc6zwM', 'Fd6AMP0YU3k', 'vtg0oqllKtI', 'L-Pv7uROcac', 'sNDoLXorAKg', 'FRgC0mKiu9c', '_v1Rv1oIEWk', 'CMIvqyokTbw', 'tVeXZ7TRzwk', '7lMtei_3Cp4', 'zqEM-HmAhvk', '-ImSbzXWMA4', 'JLkYTc6U-g0']
https://www.youtube.com/watch?v=8DcJxx5jpFQ


In [5]:
data_dict = data_dict_list[0]
data_soup = data_dict["data_soup_video_list"][0]

video_id = data_soup.select_one('meta[itemprop="videoId"]')["content"]
video_url = data_soup.select_one('[name="twitter:url"]')["content"]
published = data_soup.select_one('meta[itemprop="uploadDate"]')["content"]
title = data_soup.select_one('meta[itemprop="name"]')["content"]
img_link = data_soup.select_one('link[itemprop="thumbnailUrl"]')["href"]
description = data_soup.select_one('meta[itemprop="description"]')["content"]
playlist_id = data_dict["playlist_id"]
playlist_title = data_dict["playlist_title"]

tag_pattern = r'"shortDescription":"(#.*?)\\n'
data_soup_str = str(data_soup)
try:
    tag_list = re.findall(tag_pattern, data_soup_str)[0]
    tag_list = tag_list.split("#")
    tag_list.remove("")
except:
    tag_list = []

data = {
    "channel_name":channel_name,
    "channel_url": channel_url,
    "channel_id": channel_id,
    "video_id": video_id,
    "video_url": video_url,
    "published": published,
    "title": title,
    "img_link": img_link,
    "description": description,
    "tag_list": tag_list,
    "playlist_id": playlist_id,
    "playlist_title": ""
}

data

IndexError: list index out of range

In [ ]:
data_soup

In [23]:
url = "https://www.youtube.com/watch?v=i5R3kXZ2QZA"
resp = requests.get(url, channel_name)
resp.text
data_soup = BeautifulSoup(resp.text, 'html.parser')
video_title = data_soup.select_one('meta[name="title"]')["content"]
query_url = f"https://www.youtube.com/results?sp=mAEB&search_query={video_title}+{channel_name}"
resp = requests.get(query_url)
data_soup = BeautifulSoup(resp.text, 'html.parser')
data_soup_str = str(data_soup)
playlist_pattern = r'{"playlistRenderer":{"playlistId":"(.*?)"'
playlist_id = re.findall(playlist_pattern, data_soup_str)[0]
playlist_id

'PLl-MS2zMZDrNMYUlZQmJ3NVu4BWZwAtkb'

In [73]:
class RequestsCrawler:
    def __init__(self):
        print("===crawler init ===")

    def get_yt_playlist(self, url, channel_name):
        resp = requests.get(url)
        data_soup = BeautifulSoup(resp.text, 'html.parser')
        video_title = data_soup.select_one('meta[name="title"]')["content"]
        query_url = f"https://www.youtube.com/results?sp=mAEB&search_query={video_title}+{channel_name}"
        resp = requests.get(query_url)
        data_soup = BeautifulSoup(resp.text, 'html.parser')
        data_soup_str = str(data_soup)
        playlist_pattern = r'{"playlistRenderer":{"playlistId":"(.*?)"'
        playlist_id = re.findall(playlist_pattern, data_soup_str)[0]
        
        return playlist_id
    
    def get_yt_channel_info(self, channel_id):
        url = f"https://www.youtube.com/feeds/videos.xml?channel_id={channel_id}"
        resp = requests.get(url)
        data_soup = BeautifulSoup(resp.content, 'xml')
        channel_name = data_soup.select_one("feed > title").text
        channel_url = data_soup.select_one('feed > link[rel="alternate"]')["href"]
        channel_feeds_url = url
        channel_author_name = data_soup.select_one('author > name').text
        channel_published = data_soup.select_one('feed > published').text

        channel_data = {
            "channel_id": channel_id,
            "channel_name": channel_name,
            "channel_url": channel_url,
            "channel_author_name": channel_author_name,
            "channel_feeds_url": channel_feeds_url,
            "channel_published": channel_published
        }
        
        return channel_data
    
    def get_yt_playlist_info(self, playlist_id):
        url = f"https://www.youtube.com/watch?v=&list={playlist_id}"
        resp = requests.get(url)
        data_soup = BeautifulSoup(resp.text, 'html.parser')
        data_soup_str = str(data_soup)
        playlist_title_pattern = r'"titleText":{"runs":\[{"text":"(.*?)"'
        playlist_title = re.findall(playlist_title_pattern, data_soup_str)[0]
        
        playlist_data = {
            "playlist_id": playlist_id,
            "playlist_title": playlist_title
        }
        
        return playlist_data

In [74]:
crawler = RequestsCrawler()
playlist_id = "PLl-MS2zMZDrN2n5kLC0hLNl33je28GAtV"
crawler.get_yt_playlist_info(playlist_id)

===crawler init ===


{'playlist_id': 'PLl-MS2zMZDrN2n5kLC0hLNl33je28GAtV',
 'playlist_title': '【活動教學影片】菁鷹特務學院-皇家的祕寶'}

In [76]:
req_crawler = RequestsCrawler()
video_url = "https://www.youtube.com/watch?v=WoLjMZ-d9a8"
channel_name = "LIS情境科學教材"
playlist_id = req_crawler.get_yt_playlist(video_url,channel_name)

===crawler init ===


IndexError: list index out of range